<H1>FBN1: acromicric and geleophysic dysplasia (Le Goff, 2011)</H1>
<p>Extract phenopackets from the clinical data in <a href="https://pubmed.ncbi.nlm.nih.gov/21683322/" target="__blank">Le Goff et al (2011)</a>.</p>

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.57.dev0


In [2]:
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_hpo(release='v2024-01-16')

In [3]:
PMID = "PMID:21683322"
title = "Mutations in the TGFβ binding-protein-like domain 5 of FBN1 are responsible for acromicric and geleophysic dysplasias"
cite = Citation(pmid=PMID, title=title)

metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo.version)
print(f"HPO version {hpo.version}")

HPO version 2024-01-16


In [4]:
df = pd.read_excel('input/LeGoff_FBN1_AD_GD.xlsx')

<H2>Geleophysic dysplasia and Acromicric dysplasia: Clinical description</H2>
<p>According to Le Goff et al., Geleophysic dysplasia (GD, [MIM 231050]) and acromicric dysplasia (AD, [MIM 102370]) belong to the acromelic dysplasia group and are both characterized by severe short stature (&lt;−3 standard deviations [SD]), short hands and feet, joint limitations, and skin thickening.1
Radiological manifestations include delayed bone age, cone-shaped epiphyses, shortened long tubular bones, and ovoid vertebral bodies. GD is distinct from AD because it has an autosomal-recessive mode of inheritance, characteristic facial features—a “happy” face with full cheeks, a shortened nose, hypertelorism, a long and flat philtrum, and a thin upper lip—a progressive cardiac valvular thickening often leading to an early death, toe walking, tracheal stenosis, respiratory insufficiency, and lysosomal-like storage vacuoles in various tissues.</p> 
<p>In this notebook, we will first extract the data for Geleophysic dysplasia.</p>

<H3>Geleophysic dysplasia</H3>
<p>According to the authors, there were "Nineteen GD cases were included in the study, and they all fulfilled the diagnostic criteria, namely short stature &lt;−3 SD, short hands and feet, restricted joint mobility, characteristic facial features, and progressive cardiac involvement (Table 1, Figure 1). </p>
<p>Detailed clinical data are not available, but according to this description, we will assume that each proband had "progressive dilation and thickening of the pulmonary, aortic, and mitral valves, with stenosis of these three valves (See the <a href="https://www.ncbi.nlm.nih.gov/books/NBK11168/">GeneReview</a> entry for geleophysic dysplasia) as well as the above named facial features.</p>

In [5]:
gd_df = df[df['Diagnosis']=="GD"]
#gd_df.set_index("Family", inplace=True)
#gd_df["individual_id"] = df.index
gd_df.head(50)

,Family,Origin,Diagnosis,Age (Years),Height,Cardiac Involvement,Other,HGVS,protein
0,1,Belgium,GD,Death at 9,<−6 SD (80 cm),mitral stenosis and insufficiency,tracheotomy at 3,c.5087A>G,p.Tyr1696Cys
1,2,France,GD,18,<−6 SD (112 cm),mitral stenosis,"HTAP, respiratory insufficiency,\n\nhepatomegaly, laryngeal stenosis",c.5096A>G,p.Tyr1699Cys
2,3,Russia,GD,12,<−6 SD (106 cm),no,hepatomegaly,c.5284G>A,p.Gly1762Ser
3,4,Switzerland,GD,21,<−6 SD (116 cm),"tricuspid stenosis, mild aortic insufficiency",NaN,c.5096A>G,p.Tyr1699Cys
4,5,Russia,GD,8,−4 SD (103.5 cm),no,NaN,c.5284G>A,p.Gly1762Ser
5,6,France,GD,5.7,−4 SD (97 cm),no,laryngeal and respiratory insufficiency,c.5284G>A,p.Gly1762Ser
6,7,U.K.,GD,Death at 3,−5 SD (75 cm),no,"respiratory insufficiency, HTAP,\n\nSleep apnea",c.5087A>G,p.Tyr1696Cys
7,8,Turkey,GD,4.5,−4 SD (85 cm),mitral and tricuspide stenosis,"respiratory insufficiency, hepatomegaly, spleep apnea",c.5096A>G,p.Tyr1699Cys
8,9,Algeria,GD,Death at 4,<−6 SD (60 cm),mitral and tricuspide stenosis,"laryngeal and respiratory insufficiency, HTAP",c.5117G>A,p.Cys1706Tyr
9,10,Lebanon,GD,14,−3.5 SD (133 cm),no,–,c.5157C>G,p.Cys1719Trp


<h3>Description of GD cases</h3>
<p>Nineteen GD cases were included in the study, and they all fulfilled the diagnostic criteria, namely short stature &lt;−3 SD, short hands and feet, restricted joint mobility, characteristic facial features, and progressive cardiac involvement.</p>
<p>From this description, we will assume that all patients have the following HPO terms in addition to the terms implied by the table (which was copied with minor modifications from Table 1 of the original paper).</p>
<ol>
    <li>Short stature HP:0004322</li>
    <li>Short palm HP:0004279</li>
    <li>Short foot HP:0001773</li>
    <li>Limitation of joint mobility HP:0001376</li>
    <li>Full cheeks HP:0000293</li>
    <li>Short nose HP:0003196</li>
    <li>Hypertelorism HP:0000316</li>
    <li>Long philtrum HP:0000343</li>
    <li>Smooth philtrum HP:0000319</li>
    <li>Thin upper lip vermilion HP:0000219</li>
</ol>
<p>See above for the list of characteristic facial features that we include here.</p>    

In [6]:
column_mapper_list = list()

In [7]:
constant_features = [
    ["Short stature", "HP:0004322"],
    ["Short palm", "HP:0004279"],
    ["Short foot", "HP:0001773"],
    ["Limitation of joint mobility", "HP:0001376"],
    ["Full cheeks","HP:0000293"],
    ["Short nose", "HP:0003196"],
    ["Hypertelorism","HP:0000316"],
    ["Long philtrum", "HP:0000343"],
    ["Smooth philtrum","HP:0000319"],
    ["Thin upper lip vermilion", "HP:0000219"],
]
# the constant list can be applied to any column. We will apply it to the column "Height", 
# because all individuals have short stature
constMapper = ConstantColumnMapper(column_name="Height",term_list=constant_features)
column_mapper_list.append(constMapper)
#constMapper.preview_column(gd_df)

In [8]:
from pyphetools.creation import HpoExactConceptRecognizer

hpo_cr = HpoExactConceptRecognizer.from_hpo(hpo)

In [9]:
# Cardiac Involvement
# Note: Change entries to enable text mining
gd_df.at[1,"Cardiac Involvement"] = "Mitral stenosis; Mitral regurgitation" # was mitral stenosis and insufficiency
gd_df.at[8,"Cardiac Involvement"] = "Mitral stenosis; Tricuspid stenosis"  # was mitral and tricuspide stenosis
gd_df.at[9,"Cardiac Involvement"] = "Mitral stenosis; Tricuspid stenosis"  # was mitral and tricuspide stenosis
gd_df.at[15,"Cardiac Involvement"] = "Aortic stenosis; Mitral regurgitation; Aortic regurgitation"

#mitral and aortic valve insufficiencies
cardiacMapper = OptionColumnMapper(column_name="Cardiac Involvement",concept_recognizer=hpo_cr, option_d={})
column_mapper_list.append(cardiacMapper)
cardiacMapper.preview_column(gd_df)

,mapping,count
0,Mitral stenosis (HP:0001718) (observed),4
1,Mitral regurgitation (HP:0001653) (observed),3
2,Tricuspid stenosis (HP:0010446) (observed),3
3,Aortic regurgitation (HP:0001659) (observed),2
4,Mitral valve prolapse (HP:0001634) (observed),1
5,Aortic valve stenosis (HP:0001650) (observed),2
6,Hypertension (HP:0000822) (observed),1


In [10]:
otherMap = OptionColumnMapper(column_name="Other",concept_recognizer=hpo_cr, option_d={})
column_mapper_list.append(otherMap)
otherMap.preview_column(gd_df)

,mapping,count
0,Respiratory insufficiency (HP:0002093) (observed),5
1,Hepatomegaly (HP:0002240) (observed),5
2,Laryngeal stenosis (HP:0001602) (observed),1
3,Sleep apnea (HP:0010535) (observed),1
4,Apnea (HP:0002104) (observed),1
5,Pyloric stenosis (HP:0002021) (observed),1


<h2>Variants</h2>
<p>Note that there is an error in Table 2 of the original publication. NM_000138.5:c.5251T>G: Variant reference (T) does not agree with reference sequence (C). However, according to Clinvar the variant should be
NM_000138.5(FBN1):c.5250T>G (p.Ser1750Arg). We corrected this in the input file.</p>

In [11]:
fbn1_transcript='NM_000138.5' 
vman = VariantManager(df=gd_df, 
                      transcript=fbn1_transcript, 
                      gene_symbol="FBN1", 
                      allele_1_column_name='HGVS', 
                      individual_column_name="Family")
varMapper = VariantColumnMapper(variant_d=vman.get_variant_d(),
                                variant_column_name='HGVS', 
                                default_genotype='heterozygous')

In [12]:
ageMapper = AgeColumnMapper.by_year("Age (Years)")
encoder = CohortEncoder(df=gd_df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="Family", 
                        age_of_onset_mapper=ageMapper, 
                        sexmapper=SexColumnMapper.not_provided(),
                        variant_mapper=varMapper,
                        metadata=metadata)
omim_id = "OMIM:614185"
omim_label = "Geleophysic dysplasia 2"
gd2 = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=gd2)

In [13]:
individuals_geleophysic = encoder.get_individuals()
cvalidator = CohortValidator(cohort=individuals_geleophysic, ontology=hpo, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_html()))

In [14]:
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals_geleophysic]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
1 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5087A>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Mitral stenosis (HP:0001718)
2 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5096A>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Mitral stenosis (HP:0001718); Mitral regurgitation (HP:0001653); Respiratory insufficiency (HP:0002093); Hepatomegaly (HP:0002240); Laryngeal stenosis (HP:0001602)
3 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5284G>A (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Hepatomegaly (HP:0002240)
4 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5096A>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Tricuspid stenosis (HP:0010446); Aortic regurgitation (HP:0001659)
5 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5284G>A (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219)
6 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5284G>A (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Respiratory insufficiency (HP:0002093)
7 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5087A>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Respiratory insufficiency (HP:0002093); Sleep apnea (HP:0010535)
8 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5096A>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Respiratory insufficiency (HP:0002093); Hepatomegaly (HP:0002240); Apnea (HP:0002104)
9 (UNKNOWN; ),Geleophysic dysplasia 2 (OMIM:614185),NM_000138.5:c.5117G>A (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Limitation of joint mobility (HP:0001376); Full cheeks (HP:0000293); Short nose (HP:0003196); Hypertelorism (HP:0000316); Long philtrum (HP:0000343); Smooth philtrum (HP:0000319); Thin upper lip vermilion (HP:0000219); Mit

<H1>Acromicric dysplasia</H1>

In [15]:
ad_df = df[df['Diagnosis']=="AD"]
ad_df

,Family,Origin,Diagnosis,Age (Years),Height,Cardiac Involvement,Other,HGVS,protein
19,20,France,AD,10,−6 SD (99 cm),no,-,c.5182G>A,p.Ala1728Thr
20,21,France,AD,62,−6 SD (125 cm),no,broncho-pulmonary infection,c.5165C>G,p.Ser1722Cys
21,22-a,France,AD,10,−3 SD (121 cm),no,NaN,c.5250T>G,p.Ser1750Arg
22,22-b,France,AD,13,−3.5 SD (128 cm),no,NaN,c.5250T>G,p.Ser1750Arg
23,22-c,France,AD,40,−6 SD (128 cm),no,mother of 22 a and 22b,c.5250T>G,p.Ser1750Arg
24,23,Belgium,AD,14,<−6 SD (111 cm),no,broncho-pulmonary infection,c.5177G>T,p.Gly1726Val
25,24,Netherlands,AD,36,<−6 SD (119cm),no,"carpal tunnel syndrome, laminectomy C1-C3 for cervical spine stenosis",c.5096A>G,p.Tyr1699Cys
26,25,France,AD,13,<−6 SD (104cm),no,NaN,c.5202_5204dup,p.Gln1735dup
27,26,Italy,AD,43,<−6 SD (129cm),no,NaN,c.5273A>T,p.Asp1758Val
28,27-a,China,AD,10,−4 SD (117 cm),no,NaN,c.5099A>G,p.Tyr1700Cys


In [16]:
column_mapper_list = list()

<h3>Constant features</h3>
<p>Ten AD cases, including two familial cases, were included in the study. They all fulfilled the diagnostic criteria for AD, namely severe short stature, short hands and feet, progressively stiff joints, and characteristic facial features. AD has an autosomal-dominant mode of inheritance and is characterized by distinct facial features—a round face, well-defined eyebrows, long eyelashes, a bulbous nose with anteverted nostrils, a long and prominent philtrum, and thick lips with a small mouth—a hoarse voice, a pseudomuscular build, and distinct skeleton features, including an internal notch of the femoral head, an internal notch of the second metacarpal, and the external notch of the fifth metacarpal.</p>
<p>From this description, we will assume that all patients have the following HPO terms in addition to the terms implied by the table (which was copied with minor modifications from Table 1 of the original paper).</p>
<ol>
     <li>Short stature HP:0004322</li>
    <li>Short palm HP:0004279</li>
    <li>Short foot HP:0001773</li>
    <li>Joint stiffness HP:0001387</li>
    <li>Round face HP:0000311</li>
    <li>Long eyelashes HP:0000527</li>
    <li>Bulbous nose HP:0000414</li>
    <li>Anteverted nares HP:0000463</li>
    <li>Internal notch of the femoral head HP:0031027</li>
</ol>

In [17]:
constant_features = [
    ["Short stature", "HP:0004322"],
    ["Short palm", "HP:0004279"],
    ["Short foot", "HP:0001773"],
    ["Joint stiffness", "HP:0001387"],
    ["Round face", "HP:0000311"],
    ["Long eyelashes", "HP:0000527"],
    ["Bulbous nose", "HP:0000414"],
    ["Anteverted nares", "HP:0000463"],
    ["Internal notch of the femoral head", "HP:0031027"]
]
# the constant list can be applied to any column. We will apply it to the column "Height", 
# because all individuals have short stature
constMapper = ConstantColumnMapper(column_name="Height", term_list=constant_features)
#statureMapper.preview_column(gd_df)
column_mapper_list.append(constMapper)

In [18]:
# No cardiac involvement, in particular, no "Mitral stenosis; Tricuspid stenosis; Aortic stenosis"
constant_excluded_features = [
    ["Mitral stenosis",  "HP:0001718"],
    ["Tricuspid stenosis", "HP:0010446"],
    ["Aortic valve stenosis","HP:0001650"]
]
# the constant list can be applied to any column. We will apply it to the column "Height", 
# because all individuals have short stature
constExcludedMapper = ConstantColumnMapper(column_name="Origin",term_list=constant_excluded_features, excluded=True)
#constExcludedMapper.preview_column(gd_df["Origin"])
column_mapper_list.append(constExcludedMapper)

In [19]:
otherMap = OptionColumnMapper(column_name="Other",concept_recognizer=hpo_cr, option_d={})
column_mapper_list.append(otherMap)
otherMap.preview_column(gd_df)

,mapping,count
0,Respiratory insufficiency (HP:0002093) (observed),5
1,Hepatomegaly (HP:0002240) (observed),5
2,Laryngeal stenosis (HP:0001602) (observed),1
3,Sleep apnea (HP:0010535) (observed),1
4,Apnea (HP:0002104) (observed),1
5,Pyloric stenosis (HP:0002021) (observed),1


In [20]:
fbn1_transcript='NM_000138.5' 
vman = VariantManager(df=ad_df, 
                      transcript=fbn1_transcript, 
                      gene_symbol="FBN1", 
                      allele_1_column_name='HGVS', 
                      individual_column_name="Family")
varMapper = VariantColumnMapper(variant_d=vman.get_variant_d(),
                                variant_column_name='HGVS', 
                                default_genotype='heterozygous')

In [21]:
ageMapper = AgeColumnMapper.by_year("Age (Years)")
encoder_acromicric = CohortEncoder(df=ad_df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="Family", 
                        age_of_onset_mapper=ageMapper, 
                        sexmapper=SexColumnMapper.not_provided(),
                        variant_mapper=varMapper,
                        metadata=metadata)
omim_id = "OMIM:102370"
omim_label = "Acromicric dysplasia"
amd = Disease(disease_id=omim_id, disease_label=omim_label)
encoder_acromicric.set_disease(disease=amd)

In [22]:
individuals_acromicric = encoder_acromicric.get_individuals()
cvalidator = CohortValidator(cohort=individuals_acromicric, ontology=hpo, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_html()))

In [23]:
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals_acromicric]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
20 (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5182G>A (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); excluded: Aortic valve stenosis (HP:0001650)
21 (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5165C>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); Recurrent pneumonia (HP:0006532); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); excluded: Aortic valve stenosis (HP:0001650)
22-a (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5250T>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); excluded: Aortic valve stenosis (HP:0001650)
22-b (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5250T>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); excluded: Aortic valve stenosis (HP:0001650)
22-c (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5250T>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); excluded: Aortic valve stenosis (HP:0001650)
23 (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5177G>T (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); Recurrent pneumonia (HP:0006532); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); excluded: Aortic valve stenosis (HP:0001650)
24 (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5096A>G (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); Constrictive median neuropathy (HP:0012185); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); excluded: Aortic valve stenosis (HP:0001650)
25 (UNKNOWN; ),Acromicric dysplasia (OMIM:102370),NM_000138.5:c.5202_5204dup (heterozygous),Short stature (HP:0004322); Short palm (HP:0004279); Short foot (HP:0001773); Joint stiffness (HP:0001387); Round face (HP:0000311); Long eyelashes (HP:0000527); Bulbous nose (HP:0000414); Anteverted nares (HP:0000463); Internal notch of the femoral head (HP:0031027); excluded: Mitral stenosis (HP:0001718); excluded: Tricuspid stenosis (HP:0010446); e

In [24]:
Individual.output_individuals_as_phenopackets(individual_list=individuals_acromicric,
                                              metadata=metadata,
                                              outdir="phenopackets")

We output 13 GA4GH phenopackets to the directory phenopackets


In [25]:
Individual.output_individuals_as_phenopackets(individual_list=individuals_geleophysic,
                                              metadata=metadata,
                                              outdir="phenopackets")

We output 19 GA4GH phenopackets to the directory phenopackets
